In [ ]:
from jigsolver import Puzzle
import numpy as np
import matplotlib.pyplot as plt
from copy import copy, deepcopy

In [ ]:
img_folder = 'img/'
img = plt.imread(img_folder + "eiffel.jpg")

In [ ]:
eiffel_puzzle = Puzzle(patch_size=100)
eiffel_puzzle.create_from_img(img)
eiffel_puzzle.display()

In [ ]:
eiffel_unsolved = copy(eiffel_puzzle)
eiffel_unsolved.shuffle()

In [ ]:
some_piece = eiffel_unsolved.bag_of_pieces[0]
eiffel_unsolved.board[3,2] = some_piece
some_piece = eiffel_unsolved.bag_of_pieces[1]
eiffel_unsolved.board[2,2] = some_piece

In [ ]:
eiffel_unsolved.display()